# Semi-supervised node classification using Heterogenous Graph Neural Networks


In this tutorial, you will learn:

* Build a relational graph neural network model, a popular GNN architecture proposed by [Schlichtkrull et al.](https://arxiv.org/abs/1703.06103)
* Train the model and understand the result.

In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp

Using backend: pytorch


We first load the graph and node labels as is covered in the last session.

In [2]:
from dgl.data.rdf import AIFBDataset

dataset = AIFBDataset()
g = dataset[0]

category = dataset.predict_category
num_classes = dataset.num_classes

# obtain the training testing splits stored as graph node attributes
train_mask = g.nodes[category].data.pop('train_mask')
test_mask = g.nodes[category].data.pop('test_mask')
train_idx = torch.nonzero(train_mask, as_tuple=False).squeeze()
test_idx = torch.nonzero(test_mask, as_tuple=False).squeeze()
labels = g.nodes[category].data.pop('labels')

# split dataset into train, validate, test
val_idx = train_idx[:len(train_idx) // 5]
train_idx = train_idx[len(train_idx) // 5:]

# check cuda
device = "cuda" if torch.cuda.is_available() else "cpu"
g = g.to(device)
labels = labels.to(device)
train_idx = train_idx.to(device)
test_idx = test_idx.to(device)

Done loading data from cached files.


## Define a HeteroGraphConv model

- HeteroGraphConv is a encapsulation to run DGL NN module on heterogeneous graphs. 
- A DGL NN module has to defined per relation 𝑟.
- A reduction function to merge the results on the same node type from multiple relations.

$$
h_{dst}^{(l+1)} = \underset{r\in\mathcal{R}, r_{dst}=dst}{AGG} (f_r(g_r, h_{r_{src}}^l, h_{r_{dst}}^l))$$

See also the [link](https://docs.dgl.ai/guide/nn-heterograph.html?highlight=heterogenous%20graphs).


In [3]:
# ----------- 2. create model -------------- #
# build a two-layer RGCN model
import dgl.nn as dglnn

class RGCN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, rel_names):
        super().__init__()

        self.conv1 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(in_feats, hid_feats)
            for rel in rel_names}, aggregate='sum')
        self.conv2 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(hid_feats, out_feats)
            for rel in rel_names}, aggregate='sum')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = {k: F.relu(v) for k, v in h.items()}
        h = self.conv2(graph, h)
        return h
    

- Performs a separate graph convolution on each edge type
- Sums the message aggregations on each edge type as the final result for all node types.
- HeteroGraphConv takes in a dictionary of node types and node feature tensors as input, and returns another dictionary of node types and node features.

- Since AIFB does not have node feature we will use learnable embeddings.
- In heterogenous graphs a dictionary of embeddings is used.
- The embeddings will be updated on training.

In [4]:
class NodeEmbed(nn.Module):
    def __init__(self, num_nodes, embed_size,decice):
        super(NodeEmbed, self).__init__()
        self.embed_size = embed_size
        self.node_embeds = nn.ModuleDict()
        self.device=device
        self.num_nodes=num_nodes
        for ntype in num_nodes:
            node_embed = torch.nn.Embedding(num_nodes[ntype], self.embed_size)
            nn.init.uniform_(node_embed.weight, -1.0, 1.0)
            self.node_embeds[str(ntype)] = node_embed
    
    def forward(self):
        embeds = {}
        num_nodes=self.num_nodes
        for ntype in num_nodes:
            embeds[ntype] = self.node_embeds[ntype](torch.tensor(list(range(num_nodes[ntype]))).to(self.device))
        return embeds
    

In [5]:
num_nodes = {ntype: g.number_of_nodes(ntype) for ntype in g.ntypes}

h_hidden=16
embed = NodeEmbed(num_nodes, h_hidden,device).to(device)
model = RGCN(h_hidden, h_hidden, num_classes,g.etypes).to(device)


In [6]:
# ----------- 3. set up optimizer -------------- #

optimizer = torch.optim.Adam(itertools.chain(model.parameters(), embed.parameters()), lr=0.01)

# ----------- 4. training -------------------------------- #
all_logits = []
for e in range(50):
    # forward
    embeds = embed()
    logits= model(g,embeds)[category]
    
    # compute loss
    loss = F.cross_entropy(logits[train_idx], labels[train_idx])
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    all_logits.append(logits.detach())
    
    if e % 5 == 0:
        train_acc = torch.sum(logits[train_idx].argmax(dim=1) == labels[train_idx]).item() / len(train_idx)
        val_loss = F.cross_entropy(logits[val_idx], labels[val_idx])
        val_acc = torch.sum(logits[val_idx].argmax(dim=1) == labels[val_idx]).item() / len(val_idx)
        print("Epoch {:05d} | Train Acc: {:.4f} | Train Loss: {:.4f} | Valid Acc: {:.4f} | Valid loss: {:.4f}".
              format(e, train_acc, loss.item(), val_acc, val_loss.item()))

Epoch 00000 | Train Acc: 0.2054 | Train Loss: 2.6939 | Valid Acc: 0.3214 | Valid loss: 2.1187
Epoch 00005 | Train Acc: 0.9018 | Train Loss: 0.3326 | Valid Acc: 0.6071 | Valid loss: 1.4107
Epoch 00010 | Train Acc: 0.9554 | Train Loss: 0.1472 | Valid Acc: 0.6786 | Valid loss: 1.1779
Epoch 00015 | Train Acc: 0.9643 | Train Loss: 0.1166 | Valid Acc: 0.7500 | Valid loss: 1.1948
Epoch 00020 | Train Acc: 0.9643 | Train Loss: 0.0915 | Valid Acc: 0.8214 | Valid loss: 1.2564
Epoch 00025 | Train Acc: 0.9821 | Train Loss: 0.0703 | Valid Acc: 0.8214 | Valid loss: 1.3179
Epoch 00030 | Train Acc: 0.9821 | Train Loss: 0.0534 | Valid Acc: 0.8214 | Valid loss: 1.3900
Epoch 00035 | Train Acc: 0.9821 | Train Loss: 0.0382 | Valid Acc: 0.8214 | Valid loss: 1.5029
Epoch 00040 | Train Acc: 0.9911 | Train Loss: 0.0250 | Valid Acc: 0.7500 | Valid loss: 1.6442
Epoch 00045 | Train Acc: 1.0000 | Train Loss: 0.0151 | Valid Acc: 0.6786 | Valid loss: 1.7814


In [7]:
# ----------- 5. check results ------------------------ #
    model.eval()
    embed.eval()
    embeds = embed()
    logits= model.forward(g,embeds)[category]
    test_loss = F.cross_entropy(logits[test_idx], labels[test_idx])
    test_acc = torch.sum(logits[test_idx].argmax(dim=1) == labels[test_idx]).item() / len(test_idx)
    print("Test Acc: {:.4f} | Test loss: {:.4f}".format(test_acc, test_loss.item()))
    print()

Test Acc: 0.8333 | Test loss: 0.7807

